# Linear Regression Practice

In this lab we are going to use a 1994 dataset that has detailed prices on items sold at over 400 Burguer King, Wendy's, KFC and Roy Roger restaurants in New Jersey and Pennsivania.

Roy Roger https://en.wikipedia.org/wiki/Roy_Rogers_Restaurants


The data set has zip-code level data on various items prices, characterisitics of the zip code population.

The idea with this exercise is to see wheter fast-food restaurants charge higher prices in areas with a larger concentration of African Americans. 

### Data Set Characteristics:  
K. Graddy (1997), "Do Fast-Food Chains Price Discriminate on the Race and Income Characteristics of an Area?" Journal of Business and Economic Statistics 15, 391-401.
http://people.brandeis.edu/~kgraddy/published%20papers/GraddyK_jbes1997.pdf

    :Number of Instances: 410
    
    :Attribute Information
    
    psoda         price of medium soda, 1st wave
    pfries        price of small fries, 1st wave
    pentree       price entree (burger or chicken), 1st wave
    wagest        starting wage, 1st wave
    nmgrs         number of managers, 1st wave
    nregs         number of registers, 1st wave
    hrsopen       hours open, 1st wave
    emp           number of employees, 1st wave
    psoda2        price of medium soday, 2nd wave
    pfries2       price of small fries, 2nd wave
    pentree2      price entree, 2nd wave
    wagest2       starting wage, 2nd wave
    nmgrs2        number of managers, 2nd wave
    nregs2        number of registers, 2nd wave
    hrsopen2      hours open, 2nd wave
    emp2          number of employees, 2nd wave
    compown       =1 if company owned
    chain         BK = 1, KFC = 2, Roy Rogers = 3, Wendy's = 4
    density       population density, town
    crmrte        crime rate, town
    state         NJ = 1, PA = 2
    prpblck       proportion black, zipcode
    prppov        proportion in poverty, zipcode
    prpncar       proportion no car, zipcode
    hseval        median housing value, zipcode
    nstores       number of stores, zipcode
    income        median family income, zipcode
    county        county label
    lpsoda        log(psoda)
    lpfries       log(pfries)
    lhseval       log(hseval)
    lincome       log(income)
    ldensity      log(density)
    NJ            =1 for New Jersey
    BK            =1 if Burger King
    KFC           =1 if Kentucky Fried Chicken
    RR            =1 if Roy Rogers


In [22]:
# data modules
import numpy as np
import scipy.stats as stats
import pandas as pd

# plotting modules
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

# Stats/Regresions Packages
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

# make sure charts appear in the notebook:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

#### 1. Read the data, print the last 5 rows, perform EDA (missing values, data types, describe)

In [23]:
path_to_file = 'discrim.csv' ## Change it to your path
fast_food = pd.read_csv(path_to_file)
fast_food.head()

,psoda,pfries,pentree,wagest,nmgrs,nregs,hrsopen,emp,psoda2,pfries2,...,county,lpsoda,lpfries,lhseval,lincome,ldensity,NJ,BK,KFC,RR
0,1.12,1.06,1.02,4.25,3.0,5.0,16.0,27.5,1.11,1.11,...,18,0.113329,0.058269,11.90699,10.70401,8.301521,1,0,0,1
1,1.06,0.91,0.95,4.75,3.0,3.0,16.5,21.5,1.05,0.89,...,18,0.058269,-0.094311,11.90699,10.70401,8.301521,1,1,0,0
2,1.06,0.91,0.98,4.25,3.0,5.0,18.0,30.0,1.05,0.94,...,12,0.058269,-0.094311,12.03884,10.62532,9.341369,1,1,0,0
3,1.12,1.02,1.06,5.00,4.0,5.0,16.0,27.5,1.15,1.05,...,10,0.113329,0.019803,12.05292,10.82707,9.029418,1,0,0,1
4,1.12,NaN,0.49,5.00,3.0,3.0,16.0,5.0,1.04,1.01,...,10,0.113329,NaN,12.42561,11.18840,6.579251,1,1,0,0


#### 2. The variable/feature psoda has eight missing observations. Replace those missing observations with the average price of soda per chain ( you will have to find which chain is missing psoda values)

In [24]:
chain_mean = fast_food.groupby(by='chain').mean()
chain_mean

,psoda,pfries,pentree,wagest,nmgrs,nregs,hrsopen,emp,psoda2,pfries2,...,county,lpsoda,lpfries,lhseval,lincome,ldensity,NJ,BK,KFC,RR
chain,,,,,,,,,,,,,,,,,,,,,
1,1.054217,0.918970,0.942216,4.530683,3.671429,3.355828,16.002924,19.994012,1.051056,0.934088,...,13.549708,0.050266,-0.088409,11.869696,10.736933,7.897415,0.795322,1.0,0.0,0.0
2,1.007500,0.865513,2.465769,4.594675,2.962025,3.818182,10.800000,9.546875,0.999615,0.873333,...,13.225000,0.004175,-0.154168,11.694316,10.629569,7.988640,0.850000,0.0,1.0,0.0
3,1.106042,1.004505,1.127097,4.691354,3.367347,4.868132,16.010101,19.272727,1.089570,1.044286,...,13.656566,0.098635,0.000804,11.910038,10.798163,8.006188,0.828283,0.0,0.0,1.0
4,0.971000,0.877797,1.195500,4.758929,3.406780,2.035088,12.241667,19.112069,1.016429,0.886667,...,14.550000,-0.032304,-0.135102,11.757048,10.664218,8.016217,0.750000,0.0,0.0,0.0


In [81]:
chain_mask = (fast_food['chain']==3)&(fast_food['psoda'].isnull())

In [84]:
fast_food.loc[chain_mask,'psoda']=1.106042
fast_food[chain_mask]

,psoda,pfries,pentree,wagest,nmgrs,nregs,hrsopen,emp,psoda2,pfries2,...,county,lpsoda,lpfries,lhseval,lincome,ldensity,NJ,BK,KFC,RR
183,1.106042,NaN,NaN,4.75,3.0,3.0,16.0,12.0,1.01,1.01,...,6,NaN,NaN,11.61005,10.79084,7.569412,1,0,0,1
283,1.106042,NaN,NaN,4.25,3.0,7.0,17.0,27.5,1.05,1.00,...,17,NaN,NaN,11.88242,10.85027,7.695303,1,0,0,1
310,1.106042,NaN,NaN,NaN,4.0,5.0,12.5,20.0,1.16,1.06,...,18,NaN,NaN,12.14632,11.06515,7.581720,1,0,0,1


#### 3. In one graph, plot the distribution of 'psoda' for each chain (4 histograms in one graph) add different colors per histogram, label each histogram, locate the legend on the 'upper left' side of your chart. Are there any similarities or differences in their distributions?



#### 4. The variable income has one missing value. First, identify to what state "NJ=1 or PA=2" this missing value belongs to. Then drop the "row " that corresponds to this missing value fast_food.drop("row_number", axis=0)

In [86]:
fast_food[fast_food['income'].isnull()]
fast_food.drop(384,axis=0)

,psoda,pfries,pentree,wagest,nmgrs,nregs,hrsopen,emp,psoda2,pfries2,...,county,lpsoda,lpfries,lhseval,lincome,ldensity,NJ,BK,KFC,RR
0,1.12,1.06,1.02,4.25,3.0,5.0,16.0,27.50,1.11,1.11,...,18,0.113329,0.058269,11.90699,10.70401,8.301521,1,0,0,1
1,1.06,0.91,0.95,4.75,3.0,3.0,16.5,21.50,1.05,0.89,...,18,0.058269,-0.094311,11.90699,10.70401,8.301521,1,1,0,0
2,1.06,0.91,0.98,4.25,3.0,5.0,18.0,30.00,1.05,0.94,...,12,0.058269,-0.094311,12.03884,10.62532,9.341369,1,1,0,0
3,1.12,1.02,1.06,5.00,4.0,5.0,16.0,27.50,1.15,1.05,...,10,0.113329,0.019803,12.05292,10.82707,9.029418,1,0,0,1
4,1.12,NaN,0.49,5.00,3.0,3.0,16.0,5.00,1.04,1.01,...,10,0.113329,NaN,12.42561,11.18840,6.579251,1,1,0,0
5,1.06,0.95,1.01,4.25,4.0,4.0,15.0,17.50,1.05,0.94,...,18,0.058269,-0.051293,11.90497,10.70358,8.394799,1,1,0,0
6,1.17,0.95,0.95,4.65,3.0,2.0,16.0,22.50,1.05,0.94,...,10,0.157004,-0.051293,12.26764,11.03579,7.892826,1,1,0,0
7,1.17,1.02,1.06,4.50,3.0,5.0,17.0,18.50,1.11,1.06,...,24,0.157004,0.019803,12.20005,10.89033,8.764834,1,0,0,1
8,1.18,1.02,1.06,NaN,4.0,4.0,17.0,17.00,1.10,1.01,...,10,0.165514,0.019803,11.55022,10.35182,9.819453,1,1,0,0
9,1.17,1.12,1.02,4.25,3.0,5.0,18.0,27.00,1.11,1.12,...,10,0.157004,0.113329,11.55022,10.35182,9.819453,1,0,0,1


#### 5. Graph the distribution of psoda for the entire dataset and include a line for the average in your graph.

#### 6. Find the average values of _prpblck_ and _income_ in the sample, along with their standard deviations. Can you infer the units of measurement of these two variables (Get used to do this, these are your baseline values)

### Linear Regressions

#### 7.  Consider a model to explain the price of soda _psoda_, in terms of the proportion of the African American population and the median income
    
    psoda = β0 + β1prpblck + β2income + e


In [ ]:
# Define your linear model 


# Define the target variable, called it y


# Define your predictors, called them X


# Print the shapes of your y and X



In [ ]:
# Fit your model


# Predict your y, call them predictions, print the shape of predictions
# Print the shape of predictions


In [ ]:
## Is there a difference between the means for actual values (psoda) and your predictions


In [ ]:
## Construct a scatter plot of your model. Use your predicted values as your y axis and your y values as your x axis
## Print the Mean Square error see/read this link 
## http://mste.illinois.edu/patel/amar430/meansquare.html





In [ ]:
## What is the coefficient of determination (R^2) of the prediction. In other words,what is the accuracy of your model?


In [ ]:
## Print the estimated coefficients of your model.


In [ ]:
## What is the intercept of your model.


In [ ]:
## Write your results in equation form, include the sample size and R^2
## Interpret the coefficient on prpblck

### 8.  Compare the estimate from question 7 with a simple regression estimate from _psoda_ on _prpblck_. Is the discrimination effect larger or smaller when you control/include income on your predictors?
    
    psoda = β0 + β1prpblck + e

In [ ]:
# Define your predictors and called them X


# Print the shapes of your y and X


In [ ]:
# Fit your model


In [ ]:
# find the predictors and called them predictions
# print the shape of predictions


In [ ]:
## Is there a difference between the means for actual values (psoda) and your predictions


In [ ]:
## Construct a scatter plot of your model. Use your predicted values as your y axis and your y values as your x axis
## Print the Mean Square error 



In [ ]:
## What is the accuracy of your model?


In [ ]:
## Print the estimated coefficients for the linear regression problem


In [ ]:
## What is the intercept of your model.


In [ ]:
## Write your results in equation form, include the sample size and R^2

#### 9.  Now Use StatsModel and repeat questions 7 and 8
http://statsmodels.sourceforge.net/devel/example_formulas.html

##### Helpful Notes to Keep in mind:
1. P value for a coefficient says nothing about the size of the effect that variable is having on your dependent variable - it is possible to have a highly significant result (very small P-value) for a miniscule effect.
2. With a P value of 5% (or .05) there is only a 5% chance that results you are seeing would have come up in a 
random distribution, so you can say with a 95% probability of being correct that the variable is having some effect, assuming your model is specified correctly.
3. In simple or multiple linear regression, the size of the coefficient for each independent variable gives you 
the size of the effect that variable is having on your dependent/target (y) variable, and the sign on the coefficient 
(positive or negative) gives you the direction of the effect. In regression with a single independent/predictor variable, the coefficient tells you how much the dependent variable is expected to increase (if the coefficient is positive) or decrease (if the coefficient is negative) when that independent variable increases by one unit. 
4. In regression with multiple independent/predictors variables, the coefficient tells you how much the dependent variable is expected to increase when that independent variable increases by one unit, "_holding all the other independent variables constant_". Remember to keep in mind the units which your variables are measured in.

In [ ]:
import statsmodels.formula.api as smf


#### 10. There is one fundamental step that I have "purposely" left out? Can you guess what this step is, and implement it. Use Matplotlib and Seaborn (regplot) to graph hem.

### Before you move to the next question, read this article "Why I'm not a fan of R-Squared"
http://www.johnmyleswhite.com/notebook/2016/07/23/why-im-not-a-fan-of-r-squared/

### Bonus
#### 11. Report the estimates of the follwing model (use sklearn):
    log(psoda) = β0 + β1prpblck + β2log(income) + e


In [ ]:
## If there are missing observations on any of the log variables, would you dropped them
## from the dataset, if so how does this impact your calculations?
## Fill the missing values of lpsoda with 0


In [ ]:
## Plot a histogram that will have 2 histograms showing the distribution of psoda and lpsoda
## Are they the same?


In [ ]:
## Plot a histogram of the variable log of income (lincome)


In [ ]:
## Plot a histogram of the variable income 


In [ ]:
## Can you explain what log is doing to the variables (income, and price of soda (psoda))

In [ ]:
## In one graph use seaborn regplot to graph
## graph psoda, vs lincome
## graph lpsoda vs income
## graph lpsoda vs lincome


In [ ]:
## graph lpsoda vs income


In [ ]:
# graph lpsoda vs lincome


In [ ]:
# Define your linear model 


# Define the target variable, called it y



# Define your predictors, called them X


# Print the shapes of your y and X




In [ ]:
# Fit your model


# Predict your y, call them predictions, print the shape of predictions
# Print the shape of predictions


In [ ]:
## Graph your predicted and y values


In [ ]:
## Print the coefficients, and R2, interpret your result 
## See/read this link for interpretation of results/coeficients: 
## http://www.ats.ucla.edu/stat/mult_pkg/faq/general/log_transformed_regression.htm



## Do you think that is model is more appropriate? 

In [ ]:
## Now add the variable prppov to the regression in question 11, What happened with prpblck?

# Define your linear model 


# Define the target variable, called it y



# Define your predictors, called them X


# Print the shapes of your y and X


In [ ]:
# Fit your model


# Predict your y, call them predictions, print the shape of predictions
# Print the shape of predictions

## Print the coefficients, and R2, interpret your result


In [ ]:
## Find the correlatoion between log(income) and prpov. Is it close to what you expected?


In [ ]:
## What are your thoughts on this statement:
## "Because log(income) and prppov are so highky correlated, they have no business in being 
## included in the same regression"